In [ ]:
import pandas as pd
import numpy as np
from modelling.sklearn_dataload import DataPrep
from modelling.sklearn_pipeline import Pipeline, model_exec
from dataprep.data_prep import TextPrep
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
%load_ext autoreload
%autoreload 2

In [12]:
grid = {
            'LR': {
                'type': LogisticRegression(solver='liblinear'),
                'grid': {
                    'penalty': ['l1','l2'], 
                    'C': [0.00001, 0.0001, 0.001, 0.01, 0.1, 1, 10]
                    }
                },
            'NB': {
                'type': MultinomialNB(),
                'grid': {
                    'alpha' : [1.0, 0.1, 0.01]
                }
            },
            'RF': {
            'type': RandomForestClassifier(n_estimators=50, n_jobs=-1),
            'grid': {
                'n_estimators': [10,100], 
                'max_depth': [5,50], 
                'max_features': ['sqrt','log2'],
                'min_samples_split': [2,10]
            }
        }  
}

run_type = 'build'
scoring = 'auc-roc'
score_k_val = 0.5 # for test data only

In [3]:
TRAIN = '../dataset/train.csv'

In [4]:
dp = DataPrep(text_col='comment_text', label_col='target')
tp = TextPrep()

In [6]:
print('LOADING 1% OF DATA')
df = dp.load_data(TRAIN, ['target'], 1, 1234)
df = dp.binarize_label(df, 0.5)
text = dp.get_text(df)
print('DATASET LEN: {}'.format(len(df)))

print('SPLITTING INTO X, y')
X, y = dp.split_X_y(df)

print('PRELIMINARY TEXT PROCESSING')
clean_data = text.apply(tp.clean, rmCaps = True, 
                        mapPunct=True, clSpecial=True, 
                        spCheck=False, rmStop=False, 
                        stem=True, mpContract=True)

print('CREATING TFIDF FEATURES')
tfidf = dp.create_text_feats(clean_data, 'tfidf', ngram_range=(1,3), max_features=10000)

print('JOINING TFIDF FEATURES')
X = dp.join_features(X, tfidf)

print('SPLITTING INTO TRAIN AND TEST')
X_train, X_test, y_train, y_test = dp.train_test_split(X, y, test_size=.2, train_size=.8, random_state=1234, stratify=None)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

LOADING 1% OF DATA
LOADING FEATURE STAGING DATA
DATASET LEN: 18049
SPLITTING INTO X, y
PRELIMINARY TEXT PROCESSING
CREATING TFIDF FEATURES
JOINING TFIDF FEATURES
droppingxx
SPLITTING INTO TRAIN AND TEST
(14439, 9999) (3610, 9999) (14439,) (3610,)


In [14]:
iteration_name = 'text_fts_auc_roc'
model_exec(run_type, iteration_name, grid, 
           scoring,score_k_val, X_train, y_train, X_test, y_test)

The path of the model object used for this run is found at 
models_store
The scoring function used for this run is: auc-roc
The probabilistic threshold at which the score is calculated is 0.5
The parameter grid used for the search is: 
{'LR': {'type': LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False), 'grid': {'penalty': ['l1', 'l2'], 'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10]}}, 'NB': {'type': MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True), 'grid': {'alpha': [1.0, 0.1, 0.01]}}, 'RF': {'type': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
    